In [1]:
import BioSimSpace as BSS

INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
pdb = '6rna'

Read in and visualise

In [3]:
protein = BSS.IO.readMolecules(f"../structures/{pdb}_protein_no_water.pdb")[0]
ligand = BSS.IO.readMolecules(f"../structures/{pdb}_ligand.sdf")[0]
system = protein + ligand

In [4]:
BSS.Notebook.View(system).system()

ThemeManager()

NGLWidget(gui_style='ngl')

Parameterise - we'll use the GAFF to maintain compatability with the 'Evolutionary divergence...' paper. Sage 2.2 will be a later sensitivity test. 

In [5]:
ligand_gaff = BSS.Parameters.gaff(ligand).getMolecule()

In [6]:
process = BSS.Parameters.ff14SB(protein)
protein_ff14sb = process.getMolecule()

Solvate in a rhombic docdecahedron box (we need to save on compute!)

In [7]:
# Get the minimium and maximum coordinates of the bounding box that
# minimally encloses the protein.
box_min, box_max = protein.getAxisAlignedBoundingBox()

# Work out the box size from the difference in the coordinates.
box_size = [y - x for x, y in zip(box_min, box_max)]

# How much to pad each side of the protein? (Nonbonded cutoff = 10 A)
padding = 12 * BSS.Units.Length.angstrom

# Work out an appropriate box. This will used in each dimension to ensure
# that the cutoff constraints are satisfied if the molecule rotates.
box_length = max(box_size) + 2 * padding

box, angles = BSS.Box.rhombicDodecahedronSquare(box_length)
print(box, angles)

[84.7200 A, 84.7200 A, 84.7200 A] [120.0000 degrees, 120.0000 degrees, 90.0000 degrees]


In [8]:
system = protein_ff14sb + ligand_gaff

In [9]:
solvated = BSS.Solvent.tip3p(molecule=system, box=box, angles=angles, ion_conc=0.14)

Check what ions have been added, and the total charge. 

In [10]:
# Search for all free ions. As a simple search, we look for all molecules
# that only contain a single atom.
search = solvated.search("not mols with atomidx 2")

# Print all ions and their charge.
for ion in search.atoms():
    print(f"element = {ion.element()}, charge = {ion.charge()}")

element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sodium (Na, 11), charge = 1.0000 |e|
element = Sod

In [11]:
print(f"solvated = {solvated.charge()}, protein = {protein_ff14sb.charge()}, ligand = {ligand_gaff.charge()}")

solvated = -1.2457e-07 |e|, protein = -3.0000 |e|, ligand = -3.8858e-16 |e|


In [12]:
view = BSS.Notebook.View(solvated)
view.system()

NGLWidget(gui_style='ngl')

In [14]:
BSS.IO.saveMolecules(f"{pdb}/solvated", solvated, ["prm7", "rst7"])

['/home/lex/Research/kinase_intermediate_state/abfe/6rna/solvated.prm7',
 '/home/lex/Research/kinase_intermediate_state/abfe/6rna/solvated.rst7']